# Introduction
This note evaluate all-MiniLM-L6-v2 with llama.cpp

# Use pre-complie llama.cpp

In [1]:
!cp -r /kaggle/input/llama-cpp-binary-compiled-with-gpu/llama.cpp /kaggle/working/

Cloning into 'llama.cpp'...
remote: Enumerating objects: 34888, done.
remote: Counting objects: 100% (6871/6871), done.
remote: Compressing objects: 100% (431/431), done.
remote: Total 34888 (delta 6647), reused 6531 (delta 6436), pack-reused 28017 (from 1)
Receiving objects: 100% (34888/34888), 58.34 MiB | 25.22 MiB/s, done.
Resolving deltas: 100% (25302/25302), done.
/kaggle/working/llama.cpp


In [2]:
!chmod +x -R llama.cpp
%cd llama.cpp



NOTICE: The 'server' binary is deprecated. Please use 'llama-server' instead.


In [3]:
!cp -r /usr/local/cuda-12.3/targets /usr/local/nvidia/ 

# Run perplexity
Perplexity is the main evaluation metric for large language models (LLMs). It measures how well the model predicts a given sequence of tokens. Formally, the perplexity of LLMs is the exponentiated average negative log-likelihood, which means perplexity smaller is better

Perplexity benchmark shows how accurate the model is

In [ ]:
!/kaggle/working/llama.cpp/llama-perplexity -m /kaggle/input/all-minilm-l6-v2/gguf/fp16/1/all-MiniLM-L6-v2.gguf -f /kaggle/working/llama.cpp/wikitext-2-raw/wiki.test.raw --chunks 256 > ppl_L6_fp16.log 2>&1
!/kaggle/working/llama.cpp/llama-perplexity -m /kaggle/input/all-minilm-l6-v2/gguf/q4_k_m/1/all-MiniLM-L6-v2-Q4_K_M-v2.gguf -f /kaggle/working/llama.cpp/wikitext-2-raw/wiki.test.raw --chunks 256 > ppl_Q4_v2.log 2>&1
!/kaggle/working/llama.cpp/llama-perplexity -m /kaggle/input/all-minilm-l6-v2/gguf/q4_k_m/1/all-MiniLM-L6-v2-Q4_K_M.gguf -f /kaggle/working/llama.cpp/wikitext-2-raw/wiki.test.raw --chunks 256 > ppl_Q4.log 2>&1


^C
^C


In [ ]:
!tail -6 ppl_L6_fp16.log
!tail -6 ppl_Q4_v2.log
!tail -6 ppl_Q4.log

# Benchmarking the Inference Throughput and Memory Consumption
Llama-bench shows performance and resource consumption. There are three types of test

- Prompt processing (pp): processing a prompt in batches (-p)
- Text generation (tg): generating a sequence of tokens (-n)
- Prompt processing + text generation (pg): processing a prompt followed by generating a sequence of tokens (-pg)
In the following test, we run text generation and prompt processing

In [7]:
!/kaggle/working/llama.cpp/llama-bench -m /kaggle/input/all-minilm-l6-v2/gguf/fp16/1/all-MiniLM-L6-v2.gguf -m /kaggle/input/all-minilm-l6-v2/gguf/q4_k_m/1/all-MiniLM-L6-v2-Q4_K_M-v2.gguf -m /kaggle/input/all-minilm-l6-v2/gguf/q4_k_m/1/all-MiniLM-L6-v2-Q4_K_M.gguf -mg 1 -ngl 10,20,30,35 -n 128,256,512 

ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    no
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 1 CUDA devices:
  Device 0: Tesla P100-PCIE-16GB, compute capability 6.0, VMM: yes
| model                          |       size |     params | backend    | ngl |   main_gpu |          test |                  t/s |
| ------------------------------ | ---------: | ---------: | ---------- | --: | ---------: | ------------: | -------------------: |
| bert 22M F16                   |  43.48 MiB |    22.57 M | CUDA       |  10 |          1 |         pp512 | 30833069.70 ± 6161731.76 |
src/llama.cpp:16900: GGML_ASSERT(strcmp(res->name, "result_output") == 0 && "missing result_output tensor") failed
